In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import pandas as pd
from IPython.display import HTML
import sys

project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

from src.classifiers import is_uae_real_estate_query, llm_classifier
from src.tools import visualize_tool, safe_dataframe_tool
from src.geo_tools import generate_google_maps_html

load_dotenv()

True

In [2]:
# html_code = generate_google_maps_html(df_test)
# with open("property_map_test.html", "w", encoding="utf-8") as f:
#     f.write(html_code)

In [6]:
def main_agent(query: str):
    if not is_uae_real_estate_query(query):
        return {"type": "output", "data": "This is an irrelevant question to UAE property."}

    data = safe_dataframe_tool(query)
    if isinstance(data, dict) and data.get("error"):
        return data

    action = llm_classifier(query)

    if action == "output":
        return {"type": "data", "data": data}

    elif action == "plot_stats":
        result = visualize_tool.invoke({"data": data, "query": query})
        return {"type": "plot", **result}

    elif action == "geospatial_plot":
        result = generate_google_maps_html(data)
        return {"type": "html","content": result}

    else:
        return {"error": f"Unknown action '{action}' from classifier."}


In [14]:
result = main_agent("Locate on map properties from Dubai under 2 million AED")

{'type': 'dataframe', 'value':                                                  title  \
2            Fully Furnished | Smart Home | High Floor   
7    DSO Landscape View | x3 Parking | Post Handove...   
11                  Fully Upgraded | Luxury | High-ROI   
58   VACANT | BEST Deal on the market | Biggest layout   
61          Great ROI | 1BR Furnished | Prime Location   
..                                                 ...   
530           Canal View | Mid Floor | Spacious Layout   
537           LUXURIOUS 2 BR | READY UNIT | BEST VALUE   
547                 VACANT / ALL BRANDNEW / GUARANTEED   
548     Vacant | Great ROI | Brand New | Marina Access   
553   Vacant |Sea View |Kitchen Equipped |Mid Floor  |   

                                 displayAddress  bathrooms  bedrooms  \
2                     East 40, Al Furjan, Dubai          1         1   
7    Tria By Deyaar, Dubai Silicon Oasis, Dubai          4         2   
11   Oxford 212, Jumeirah Village Circle, Dubai         

In [11]:
df = pd.read_parquet("../data/uae_real_estate_2024_geo_ready.parquet")

In [13]:
df[df["title"]=="Luxury Living | 3BR TH | Corner | Resale"]

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude
452,Luxury Living | 3BR TH | Corner | Resale,"Al Jubail Island, Abu Dhabi",5,3,2024-08-05 11:19:43+00:00,Residential for Sale,4600000,False,sell,NaN,NO,<p>Jubail Island is a serene oasis that provid...,24.53171,54.486156


In [ ]:
with open("property_map_test3.html", "w", encoding="utf-8") as f:
    f.write(result["content"])

: 